# 计算机视觉问题模型研究综述

-----

### 1. 人脸任务
---
   - 人脸检测
      - 问题描述
      
      人脸检测问题是从图像中检测出是否有人脸（二分类）、人脸的位置信息
      
      - 如何处理大小不一致的图像
         - 滑动窗口+图像金字塔：缺点的计算复杂度太高（Cascade CNN）
         - FCN：全卷积网络能处理输入尺寸不一致的问题（DenseBox MTCNN）
      
      - 经典框架
         - VJ（Haar+SVM+Adaboost）
         - Cascade CNN
            - 滑窗+级联+CNN+多尺度融合+图像金字塔+NMS
            - 检测过程
               <img src="./img/CascadeCNN.jpg">
            - 网络结构
               12-net、24-net、48-net
               <img src="./img/CascadeCNN1.jpg">
               人脸二分类+矫正网络（校正网络负责将原来是人脸的候选框赋予某一种校正（45种））
               <img src="./img/CascadeCNN2.jpg">
               
            - 训练过程
            
               24net的训练负样本从12net错分类为人脸的背景图获取
                  
            - 问题
              1. 训练数据不平衡
              2. 滑窗方法速度慢
         - MTCNN（中科院）
            - 特点：multi-task、Cascade、FCN、NMS、图像金字塔
         
         - Densebox（百度）
            - 特点：FCN、Upsampling、多任务（bounding box reg & face cls）、多尺度特征融合、Hard Negative Mining 
            - 网络结构
            
            <img src="./img/densebox1.png">
            
            - 检测过程
            <img src="./img/densebox3.png">
            - 训练过程
               - 难负样本挖掘
               > 通过寻找预测较差的样本来提高学习效率。经过negative mining后，分错的负样本更容易被挑出来，对于这些样本基于随机梯度下降的学习可以产生噪音小而且更为鲁棒的预测。negative mining可以通过在线提升来提高性能。在前向传递过程中，按分类损失将像素按降序排列，top1%的为hard-negative。在实验中，将positive于negative的比例设置在1:1.在negative samples中，一半来自于hard-negative，剩余的从非hard-negative中随机采样。为了方便，将被挑选的像素设置标记f=1.
         
 ---     
   - 人脸识别
      - 问题描述
         
         人脸识别流程：人脸检测->人脸对齐->特征提取->分类
         
         核心的问题是人脸的特征提取
      
      - 经典框架
         - DeepFace（Facebook）
            - 特点
              
            - 检测流程
               对齐后人脸(固定大小)->4096维特征->向量距离计算相似度
               <img src="./img/deepface1.png">
         - DeepID_1
            - 特点:多图像块（裁剪五官提取每块特征、拼接）
            - 检测流程
            <img src="./img/deepid1.png">
            
            分类选择神经网络检测两个图像的特征向量是否属于同一个人的
            
         - DeepID_2
            - 特点：特征提取部分的损失函数（不同的两个人差异最大化+同一个人的不同图像特征差异最小化）
            - 检测流程
            <img src="./img/deepid.png" style="background:#fff;width:400px">
         - DeepID_3
   

---
### 2. 通用目标检测
---
   - 难点
      1. 目标尺寸和图像尺寸不固定
      2. 多目标问题（图片中出现多目标）
      
   - 非deep learning的方法
      - selective search
      - NMS
   - 经典框架
      - 基于分类的检测算法
         - RCNN
            1. 特点
               - 1. 计算速度非常慢
               - 2. 输出尺度固定，窗口必须进行形变
               
            2. 检测流程：
               1. 输入图片通过Selective Search给出约2k的建议图像
               2. 将建议图像做形变变为同一size送入CNN（AlexNet）提取特征向量
               3. 使用SVM对特征向量进行多分类
               4. refine bounding box & NMS
         - SPP
             - SPP层：空间金字塔池化 将原始图像池化为a*a大小的feature map
                1. 多尺度池化 a取1、3、5
             1. 共享卷积计算：RCNN的做法是得到RegionProposal区域组，然后每个RP分别做裁剪、缩放、卷积、FC；这样处理卷积会计算很多次；共享卷积先得到RegionProposal，然后对整张图片做卷积，然后再根据RP进行裁剪，这样能减少卷积计算量（**共享卷积**）
             <img src="./img/spp.png" style="background:#fff;width:400px">
             <img src="./img/sppnet.png" style="background:#fff;width:400px">
             2. 也需要Selective Search进行边框的初步筛选
         - Fast RCNN
             1. 使用ROI池化层替代SPP层（单一尺度池化）
             2. 使用CNN进行 提取特征+分类+bbox Regression
            
         - Faster RCNN
            1. 将候选区域生成纳入神经网络学习的过程
            2. 在fastrcnn的基础上进行改进
            3. RPN网络：进行候选区域生成
              RPN--全卷积网络实现对
              > 原文中RPN网络为CNN后面接一个3*3的卷积层，再接两个1*1的卷积层(原文称这两个卷积层的关系为sibling)，其中一个是用来给softmax层进行分类，另一个用于给候选区域精确定位。
              
                  这里的分类是指目标、背景二分类
                  
                  RPN层也需要单独训练

            4. anchor 锚点
            > anchors可以理解为一些预设大小的框，anchors的种类用k表示，在原文中k=9，由3种面积(128* 128,256* 256,512* 512)和3种长宽比(1:1,1:2,2:1)组成，这里anchors的大小选取是根据检测时的图像定义，在检测时会将最小边缩放到600，最大边不超过1000

        <img src="./img/fasterrcnn.jfif" style="background:#fff;width:400px">
        
        算法比较
        
        <img src="./img/rcnn.jfif" >
<br>
<br>
      - 基于回归的检测算法
          前面的算法都是two-stage方法，先找出框、再分类;
          基于回归的检测算法不需要生成候选区域，直接产生矩形框位置和类别
         - YOLO
            you only look once
               v1、v2、v3、yolo9000
               关键词：网格划分、NMS
               预测量：S x S x (5* B+C) 
               解释： 全卷积网络，输出的维度是(S,S,5B+C)
               5表示(w,h,x,y)+confident
               confident的计算公式 isobj* IOU
               B表示每个格子预测B个框
               C是类别数
            <img src="./img/yolo.png" style="background:#fff;width:400px">
            网络结构
            <img src="./img/yolonet.png" style="background:#fff;width:400px">
         - SSD
         



3. 视频

4. 3D